In [41]:
import spacy
spacy.load('en')
from spacy.lang.en import English

import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer

from gensim import corpora
import pandas as pd
import random


In [34]:
# Tokenize
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [35]:
nltk.download('wordnet')

def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [40]:
text_data = []
df = pd.read_csv('posts_data.csv')
data = df['Body']
data

0       Does anyone get blisters on their feet from wa...
1       It's been a few months since that whole sexual...
2       I think the reason why we really hate Filipino...
3       Tangina this OBF wifi is pure shit. \r\nTo all...
4       Sometimes mood: puTanGinAaaAAaAa\r\n\r\n24/7 m...
5       The Marisol PA announcements really need more ...
6       https://tonite.abante.com.ph/ateneo-student-gi...
7                   GRABE ANG LUBAK SA COVERED COURTS WTF
8       I'm free-falling and I'm not sure if you're th...
9       I thought that the east carpark is strictly fo...
10      Why is “anti-poor” considered a bad word in po...
11      why do none of the sockets in the old lib work...
12      Stop using mental health to complain about sch...
13      To #ADMUFreedomWall17019 and #ADMUFreedomWall ...
14      i dreamt that this girl from my org and i were...
15      What is the least painful way to kill yourself...
16      Okay so I haven't had any sleep and was wander...
17      any ti

In [50]:
for line in data:
    tokens = prepare_text_for_lda(line)
    if random.random() > .99:
        print(tokens)
        text_data.append(tokens)    
    print('Original Sentence:', line)
    print('Tokens:', tokens)
    print()

Original Sentence: Does anyone get blisters on their feet from walking so much around school?
Tokens: ['anyone', 'blister', 'walking', 'around', 'school']

Original Sentence: It's been a few months since that whole sexual harassment scandal rocked ateneo. The silence is deafening. Some of the profs who have been so vocal about issues like martial law etc never spoke up about this particular issue. And now they're opening up about that fucking upsilon scandal like it's the worst shit ever? Fuck you hypocrites.
Tokens: ['month', 'since', 'whole', 'sexual', 'harassment', 'scandal', 'rock', 'ateneo', 'silence', 'deafen', 'prof', 'vocal', 'issue', 'martial', 'never', 'spoke', 'particular', 'issue', 'opening', 'fucking', 'upsilon', 'scandal', 'worst', 'hypocrite']

Original Sentence: I think the reason why we really hate Filipino in college is not because of the language, but the topics in it. I’d understand if Filipino was about learning how to speak deeper Tagalog, learning like street tag

I feel you guys. I’m an oldie of the org I assume you are referring to and to be honest I only remained a member because a) I still somehow love my work, and b) I still have very very good friends in the org. Yes they are VERY cliquish. What completely sucks is that there is a sense of “coldness” between the cast and the production teams. Hell cast members don’t even say hi to you when you say hi to them in the theater. For fuck’s sake one time, after I’d just finished doing the show for lights, I stepped out and they told me “Thank you for watching!” as of I were part of the audience. They didn’t even know I’d been there for literally almost every show. This was done to me 4 TIMES. 4 fucking times like holy shit di niyo ba ako kilala or something I try to say hi to you people every fucking time.  To all the people who defend the org and say “Just go to HR. They’re there for a reason,” this isn’t a simple “HR” problem. There’s a problem with how the org has been systematically. There i

Tokens: ['restless', 'couple', 'week', 'super', 'nervous', 'fidgety', 'since', 'stop', 'reply', 'message', 'often', 'studying', 'friend', 'space', 'matter', 'annoying', 'thought', 'linger', 'interest', 'anymore', 'rarely', 'reply', 'academic', 'space']

Original Sentence: Hayss. This group.
Tokens: ['hays', 'group']

Original Sentence: i'd take photos of the Carrie photographer but photography isn't allowed daw in the theater :(
Tokens: ['photo', 'carrie', 'photographer', 'photography', 'allow', 'theater']

Original Sentence: people trying to save endangered species want to see the animals fuck
Tokens: ['people', 'try', 'endanger', 'species', 'animal']

Original Sentence: What I expected from Filipino classes:

A great class that focuses on Filipino culture, linguistics, history and modern politics, educating students with solid information worthy of one of the top schools in the country. What I got: A teacher which uses 1 hour to rant about that "you gotta work work" Gatorade ad, sayi

THIS POST WAS MADE TO WARN PEOPLE ABOUT NETWORKING AND HOW FRAUDULENT THEIR REPS ARE I know it's not good to generalize but there was this brand (rhymes with MuSkeen) who invited me for a meeting about a job opening in my field. Already skeptical about it, I told them that I'll think about it. Joke's on me for worrying to come off as impolite if I say no.  When the day came that he followed up for my confirmation, I straight up asked the person if it was a network marketing event. He literally said NO THAT IT WASNT. Believing him, I went to the meeting because I trusted him. THAT'S WHERE I MADE THE BIGGEST MISTAKE.  The speakers of the event were going on about how "Network Marketing was the best way to earn money" UGH FUQ IVE BEEN LIED TO AND I FELL FOR IT  I'm already laughing at myself for falling for this. But I'm sharing this experience to y'all AS A CAUTIONARY TALE to avoid people messaging you about a "business venture" or a "business opportunity", take everything with a grain o

Tokens: ['please', 'downplay', 'course', 'management', 'basic', 'course', 'someone', 'calling', 'mediocre', 'grade', 'happiness', 'everyone', 'family', 'smart', 'rather', 'active', 'friend', 'judging', 'another', 'support', 'growth', 'happiness', 'instead']

Original Sentence: charot is good for your eyes
Tokens: ['charot']

Original Sentence: 19 na taong gulang pero isip bata pa rin
Tokens: ['taong', 'gulang']

Original Sentence: So I heard that the AEGIS shoot for ICA, St. Jude and Grace are all taking place at the same time

ISO, prepare for the Chinese invasion
Tokens: ['hear', 'aegis', 'shoot', 'grace', 'taking', 'place', 'prepare', 'chinese', 'invasion']

Original Sentence: my mood rn: mood
Tokens: []

Original Sentence: tangina mo naman
Tokens: ['tangina', 'naman']

Original Sentence: Is it normal to doubt ur man kapag may nag notif about sa grindr and other shit? sabi niya hindi na daw siya active pero why won't he delete those fucking apps.
tapos he said "kung nag ddoubt ka ib

...

Tokens: ['female', 'psych', 'friend', 'single', 'ready', 'mingle', 'lezgo']

Original Sentence: Sarap pala ng feeling na crush ka rin ng crush mo
Tokens: ['sarap', 'feeling', 'crush', 'crush']

Original Sentence: Fuck Upsilon.
Tokens: ['upsilon']

Original Sentence: who has masterposts/lists of printing areas and nice bathrooms on campus? 
-freshie
Tokens: ['masterposts', 'list', 'printing', 'area', 'bathroom', 'campus', '-freshie']

Original Sentence: do you ever open the submit link then stare at the submission box for a few minutes then say "wag nalang" coz you don't know how to put to words how sad you are :(
Tokens: ['submit', 'stare', 'submission', 'minutes', 'nalang', 'words']

Original Sentence: All these critics demanding historical accuracy and a deeper plot. Complaining about the way the LGBTQ elements of the film were handled. I don't care what they all say, I have never jammed harder in the cinema in my entire life! Bohemian Rhapsody is a damn good watch.
Tokens: ['critic'

Tokens: ['putting', 'effort']

Original Sentence: is it weird to save crushie's ig pics uwu
Tokens: ['weird', 'crushie']

Original Sentence: does anyone know how many calories a piece of boneless chicken is in SUTRA??? need sum info yo
Tokens: ['anyone', 'calorie', 'piece', 'boneless', 'chicken', 'sutra']

Original Sentence: I’m super into this guy in my class. He’s soo handsome, but he’s shorter than me. :-((( I want to be shorter for you </3
Tokens: ['super', 'class', 'handsome', 'short', ':-(((', 'short']

Original Sentence: OH WHAT A MOTIVATION THAT I GOT TO SEE (AND TALK) TO MY CRUSH BEFORE A PRESENTATION CONSTITUTING 25% OF MY GRADE
I'M HYPED!
Tokens: ['motivation', 'crush', 'presentation', 'constitute', 'grade', 'hype']

Original Sentence: Someone should make an event singing the Halo theme song in Secwalk or Red brick road
Tokens: ['someone', 'event', 'singing', 'theme', 'secwalk', 'brick']

Original Sentence: RIP DLSU Freedom Wall
Tokens: ['freedom']

Original Sentence: Anong 

Original Sentence: Okay so, I wish and hope that my parents wont see this meme. Bc istg, they'll mock me big time :(( kasi like, my "a" in parang becomes silent most of the time. So, it sounds like puhrung lagi :((((
Tokens: ['parent', 'parang', 'become', 'silent', 'sound', 'puhrung']

Original Sentence: Am I the only one with parents or relatives from other schools who generalize Ateneo as pro-Aquino? UGH IM HEATED!!! Being anti-Duterte and anti-Marcos doesnt equate to being pro-Aquino!!!! BY ANY MEANS!!! and yes, we arent influenced by the fact that the Aquinos were schooled here!!!!
Tokens: ['parent', 'relative', 'school', 'generalize', 'ateneo', 'aquino', 'heat', 'duterte', 'marcos', 'equate', 'aquino', 'means', 'influence', 'aquinos', 'school']

Original Sentence: Tbh, cookies by the bucket is overpriced and tastes like normal cookies
Tokens: ['cookie', 'bucket', 'overprice', 'taste', 'normal', 'cookie']

Original Sentence: COOKIES BY THE BUCKET WHERE U AT!!!! h m p h. >:--( x(

-

Tokens: ['anyone', 'street', 'fighter', 'looking', 'sparring', 'partner', 'guile', 'dhalsim', 'speed', 'steam', 'autumn', 'winter']



TypeError: object of type 'float' has no len()

Convert to bag of words

In [51]:

dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

Find topics

In [19]:
import gensim
NUM_TOPICS = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.030*"really" + 0.030*"never" + 0.030*"crush" + 0.016*"think"')
(1, '0.023*"pretty" + 0.022*"draining","november" + 0.022*"ideation" + 0.022*"16617","how"')
(2, '0.055*"class" + 0.038*"swear" + 0.038*"funds" + 0.021*"sorry"')
(3, '0.032*"every" + 0.018*"seeing" + 0.018*"campus" + 0.018*"kick"')
(4, '0.047*"crush" + 0.026*"really" + 0.026*"think" + 0.026*"kilala"')


pyLDAvis for visualization

In [21]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\Gab\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [25]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

FileNotFoundError: [Errno 2] No such file or directory: 'model3.gensim'